In [1]:
##Assessment 2 MLPR
import numpy as np
import matplotlib.pyplot as plt
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']
X_train.shape

(40754, 384)

In [2]:
########################## 1st question ####################################### 
print("Mean of y_train:",np.mean(y_train))
print("Standard error for y_train (for 5785 entries):",np.std(y_train[:5785], ddof=1)/np.sqrt(len(y_train[:5785])))
print("Mean of y_val (for 5785 entries):",np.mean(y_val))
print("Standard error for y_val (for 5785 entries):",np.std(y_val, ddof=1)/np.sqrt(len(y_val)))


## Constants

rm_idx0=[]    

for i in range(len(X_train[1])):
    col=X_train[:,i]
    if all(col[0]==col):
        rm_idx0.append(i)
        
## Duplicates

indices= np.unique(X_train,return_index=True,axis=1)[1]

indices=np.sort(indices)
rm_idx=list(set(range(indices[0],indices[-1]+1))-set(indices))

idx = np.unique(np.sort(rm_idx+rm_idx0))
print("                                          ")
print("Removed column indices are:",idx)


X_train = np.delete(X_train,idx,axis=1)
print("Modified shape of X_train:",X_train.shape)

X_val = np.delete(X_val, idx, axis=1)
print("Modified shape of X_val:",X_val.shape)

X_test = np.delete(X_test, idx, axis=1)
print("Modified shape of X_test:",X_test.shape)

Mean of y_train: -9.13868774539957e-15
Standard error for y_train (for 5785 entries): 0.011927303389170828
Mean of y_val (for 5785 entries): -0.2160085093241599
Standard error for y_val (for 5785 entries): 0.01290449880016868
                                          
Removed column indices are: [ 59  69  78  79 179 188 189 199 287 351 359]
Modified shape of X_train: (40754, 373)
Modified shape of X_val: (5785, 373)
Modified shape of X_test: (6961, 373)


In [3]:
## 2nd question



def fit_linreg(X, yy, alpha):
    k=len(X[1])                                 ## getting number of input features
    yy = np.concatenate((yy, np.zeros(k)))      ## adding 0_k to the y_train array
    z_k = np.sqrt(alpha) * np.eye(k)
    print(z_k.shape)
    X = np.vstack((X,z_k))  
    
    b = np.concatenate((np.ones(len(X)-k), np.zeros(k)))[:,None]

    X = np.insert(X,[0],b,axis=1)

    w_fit=np.linalg.lstsq(X, yy, rcond=None)[0]
    
    
    return w_fit[1:], w_fit[0]
    
    

In [4]:
alpha=30

ww0, bb0 = fit_linreg(X_train, y_train, alpha)

print("Bias is:",bb0)
print("Weights are:","\n",ww0)

(373, 373)
Bias is: 0.09105350649797651
Weights are: 
 [-6.08615525e-02 -1.06095667e-01  7.53031143e-02  2.78057665e-01
  2.59611794e-01  1.19816553e-01  1.05110635e-02  2.32613667e-01
 -3.17361266e-01 -7.77510933e-02 -4.17318470e-02  4.99788622e-02
 -2.21837493e-02 -2.47999863e-03 -7.29573201e-03  3.75843616e-02
  2.67414563e-02  3.32447940e-03 -8.15353393e-02  1.75498538e-01
  7.92614640e-02  2.86048166e-02 -4.09313322e-02 -6.22094524e-03
 -6.01902872e-02 -5.63195592e-02 -1.47997259e-02 -4.11106517e-01
  3.40469070e-01  2.63461215e-01 -1.35315466e-02  1.73477525e-03
 -9.11983421e-03 -6.98890681e-03  6.51972330e-02 -1.75618988e-01
 -1.79686338e-01  2.20532723e-01 -9.06442066e-02 -3.28680365e-02
 -1.80905054e-01  1.91033029e-02 -1.75010207e-01  5.31362120e-02
 -1.40996396e-01 -2.02005581e-02 -2.04166648e-02  2.13763380e-02
 -4.53472384e-01  8.70663389e-02 -9.25631074e-02 -3.52428163e-03
 -7.86791379e-02 -5.89370945e-02 -3.60229412e-02 -8.00712607e-02
  4.99929907e-02 -1.32414827e-01  1

In [5]:
## Support code
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve

def params_unwrap(param_vec, shapes, sizes):
    """Helper routine for minimize_list"""
    args = []
    pos = 0
    for i in range(len(shapes)):
        sz = sizes[i]
        args.append(param_vec[pos:pos+sz].reshape(shapes[i]))
        pos += sz
    return args


def params_wrap(param_list):
    """Helper routine for minimize_list"""
    param_list = [np.array(x) for x in param_list]
    shapes = [x.shape for x in param_list]
    sizes = [x.size for x in param_list]
    param_vec = np.zeros(sum(sizes))
    pos = 0
    for param in param_list:
        sz = param.size
        param_vec[pos:pos+sz] = param.ravel()
        pos += sz
    unwrap = lambda pvec: params_unwrap(pvec, shapes, sizes)
    return param_vec, unwrap


def linreg_cost(params, X, yy, alpha):
    """Regularized least squares cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # forward computation of error
    ff = np.dot(X, ww) + bb
    res = ff - yy
    E = np.dot(res, res) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    ff_bar = 2*res
    bb_bar = np.sum(ff_bar)
    ww_bar = np.dot(X.T, ff_bar) + 2*alpha*ww

    return E, [ww_bar, bb_bar]

def minimize_list(cost, init_list, args):
    """Optimize a list of arrays (wrapper of scipy.optimize.minimize)

    The input function "cost" should take a list of parameters,
    followed by any extra arguments:
        cost(init_list, *args)
    should return the cost of the initial condition, and a list in the same
    format as init_list giving gradients of the cost wrt the parameters.

    The options to the optimizer have been hard-coded. You may wish
    to change disp to True to get more diagnostics. You may want to
    decrease maxiter while debugging. Although please report all results
    in Q2-5 using maxiter=500.
    """
    opt = {'maxiter': 500, 'disp': False}
    init, unwrap = params_wrap(init_list)
    def wrap_cost(vec, *args):
        E, params_bar = cost(unwrap(vec), *args)
        vec_bar, _ = params_wrap(params_bar)
        return E, vec_bar
    res = minimize(wrap_cost, init, args, 'L-BFGS-B', jac=True, options=opt)
    return unwrap(res.x)


def fit_linreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(linreg_cost, init, args)
    return ww, bb

def logreg_cost(params, X, yy, alpha):
    """Regularized logistic regression cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration of fitting a similar function.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # Force targets to be +/- 1
    yy = 2*(yy==1) - 1

    # forward computation of error
    aa = yy*(np.dot(X, ww) + bb)
    sigma = 1/(1 + np.exp(-aa))
    E = -np.sum(np.log(sigma)) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    aa_bar = sigma - 1
    bb_bar = np.dot(aa_bar, yy)
    ww_bar = np.dot(X.T, yy*aa_bar) + 2*alpha*ww

    return E, (ww_bar, bb_bar)

In [6]:
ww1,bb1 = fit_linreg_gradopt(X_train, y_train, 30)
#print(bb1,"\n")
#print(ww1)

In [7]:
def rmse(pred,yy):
    return np.sqrt(np.mean((pred-yy)**2))

In [8]:
## for least square method
pred1_train = np.dot(X_train,ww0)+bb0
pred2_val = np.dot(X_val,ww0)+bb0
print("Root means square for training set(using least square method):",rmse(pred1_train, y_train))
print("Root means square for validation set(using least square method):",rmse(pred2_val, y_val))

Root means square for training set(using least square method): 0.3567565397204054
Root means square for validation set(using least square method): 0.4230521968394701


In [9]:
# for gradient method
pred1_train = np.dot(X_train,ww1)+bb1
pred2_val = np.dot(X_val,ww1)+bb1
print("Root means square for training set(using least square method):",rmse(pred1_train, y_train))
print("Root means square for validation set(using least square method):",rmse(pred2_val, y_val))

Root means square for training set(using least square method): 0.3567569385517838
Root means square for validation set(using least square method): 0.4230540100048547


In [10]:
######################  3rd question ################################
def fit_logreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb


K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

w_fit2= np.array([[0.0]* (len(X_train[1])+1)] * K)
for kk in range(K):
    labels = y_train > thresholds[kk]
    ww2, bb2 = fit_logreg_gradopt(X_train, labels, alpha=30)
    w_fit2[kk,0] = bb2
    w_fit2[kk,1:]=ww2


In [11]:
bb2 = w_fit2[:,0]
ww2 = w_fit2[:,1:]

def sigmoid(a):
    return 1 / (1+np.exp(-a))



X_train_new = sigmoid(np.dot(X_train, np.transpose(ww2))+bb2) 
X_val_new = sigmoid(np.dot(X_val, np.transpose(ww2))+bb2)


nn_ww, nn_bb = fit_linreg(X_train_new, y_train, alpha=30)

(20, 20)


In [12]:
pred1_train = np.dot(X_train_new, nn_ww) + nn_bb
pred2_val = np.dot(X_val_new, nn_ww) + nn_bb

print("Root means square for training set:",rmse(pred1_train, y_train))
print("Root means square for validation set:",rmse(pred2_val, y_val))

Root means square for training set: 0.15441150430439968
Root means square for validation set: 0.2542477298370707
